In [77]:
# Cell 0
# If at all possible please test locally or on the private tbears server. The Testnet
# is becoming cluttered with many deployments of Balanced contracts.
# Note that running on the private tbears server will require the number of top P-Reps 
# be set to 4 in the staking contract or it will fail to deploy.

network = "custom"  # set this to one of mainnet, yeouido, euljiro, pagoda, or custom

connections = {
"mainnet": {"iconservice": "https://ctz.solidwallet.io",       "nid": 1},
"yeouido": {"iconservice": "https://bicon.net.solidwallet.io", "nid": 3},
"gangnam": {"iconservice": "https://gicon.net.solidwallet.io", "nid": 3},
"hannam": {"iconservice": "https://hannam.net.solidwallet.io", "nid": 3},
"euljiro": {"iconservice": "https://test-ctz.solidwallet.io",  "nid": 2},
"pagoda":  {"iconservice": "https://zicon.net.solidwallet.io", "nid":80},
"custom":  {"iconservice": "http://18.144.108.38:9000",        "nid": 3}}

env = connections[network]

In [78]:
# Cell 1

from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from repeater import retry
from shutil import make_archive
import pickle as pkl
from datetime import datetime
from time import sleep
import json
import os

ICX = 1000000000000000000 # 10**18
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"
TEST_ORACLE = "cx61a36e5d10412e03c907a507d1e8c6c3856d9964"
MAIN_ORACLE = "cxe647e0af68a4661566f5e9861ad4ac854de808a2"
BALANCED_TEST = "hx3f01840a599da07b0f620eeae7aa9c574169a4be"

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result

In [79]:
# Cell 2

icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]

In [80]:
# Cell 3

wallet = KeyWallet.load("../../keystores/keystore_test1.json", "test1_Account")
# Balanced test wallet
with open("../../keystores/balanced_test.pwd", "r") as f:
    key_data = f.read()
btest_wallet = KeyWallet.load("../../keystores/balanced_test.json", key_data)
with open("../../keystores/staking_test.pwd", "r") as f:
    key_data = f.read()
staking_wallet = KeyWallet.load("../../keystores/staking_test.json", key_data)


In [81]:
print(wallet.get_address())
print(icon_service.get_balance(wallet.get_address()) / 10**18)

hxe7af5fcfd8dfc67530a01a0e403882687528dfcb
779046486.5654812


In [82]:
print(btest_wallet.get_address())
print(icon_service.get_balance(btest_wallet.get_address()) / 10**18)

hx3f01840a599da07b0f620eeae7aa9c574169a4be
1944235.4601872


In [83]:
user1 = KeyWallet.load("../../keystores/user1.json","HelloWorld@1234")
# btest_wallet = KeyWallet.load("./balanced_test.json","HelloWorld@1234")

print(icon_service.get_balance(user1.get_address())/10**18)
print(user1.get_address())

# test2 = hx7a1824129a8fe803e45a3aae1c0e060399546187
private = "0a354424b20a7e3c55c43808d607bddfac85d033e63d7d093cb9f0a26c4ee022"
user2 = KeyWallet.load(bytes.fromhex(private))
print(icon_service.get_balance(user2.get_address())/10**18)
print(user2.get_address())

181999.8088096
hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db
206479.709028
hx7a1824129a8fe803e45a3aae1c0e060399546187


In [117]:
dict_list = []
for i in range(0, 5):
    data = {}
    wallet2 = KeyWallet.create()
    address = wallet2.get_address()
    private_key = wallet2.get_private_key()
    data[address] = private_key
    dict_list.append(data)
with open('wallet2.json', 'a') as f:
    json.dump(dict_list, f, ensure_ascii=False, indent=4)

In [145]:
with open('wallet2.json') as f:
    data = json.load(f)

In [119]:
for dict in data:
    for items in dict.items():
        print(items[0])
        set_div_per = TransactionBuilder() \
            .from_(wallet.get_address()) \
            .to(items[0]) \
            .nid(NID) \
            .nonce(100) \
            .value(5000 * 10 ** 18) \
            .build()
        estimate_step = icon_service.estimate_step(set_div_per)
        step_limit = estimate_step + 10000000
        signed_transaction = SignedTransaction(set_div_per, wallet, step_limit)

        tx_hash = icon_service.send_transaction(signed_transaction)
        ab = get_tx_result(tx_hash)

hxc92735c9a954399c2fb722ae28bf22a10e27491e


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
hxa9e67162f41c05c651d50d1197fc8596864af801


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
hx7b4dd3efcc72c5e2b210852742593178c210742c


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
hx22722df8f35ddbe40954cb4496619ae932a4ac3b


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
hxe6645482385915b2329f444f59abf49fe5024bd7


In [9]:
contracts = {'loans': {'zip': '../../core_contracts/loans.zip', 'SCORE': 'cxe38b01784e58f72d75aa8450bdee22d62fbdb054'}, 'staking': {'zip': '../../core_contracts/staking.zip', 'SCORE': 'cxd37f220d61294c14e7d38d5e8f723c6c5a81d994'}, 'dividends': {'zip': '../../core_contracts/dividends.zip', 'SCORE': 'cx3144c3c8ea295653a8f92e83b9d6b0f2c3457e2c'}, 'reserve': {'zip': '../../core_contracts/reserve.zip', 'SCORE': 'cxf8058e6dca2dae0a7a2aa470cde7d5e91bb0c562'}, 'daofund': {'zip': '../../core_contracts/daofund.zip', 'SCORE': 'cx50f12e56855d5ad266304fec71603a0eda638735'}, 'rewards': {'zip': '../../core_contracts/rewards.zip', 'SCORE': 'cx2ef5f06ac02d861e735222cc0718c83d2419405f'}, 'dex': {'zip': '../../core_contracts/dex.zip', 'SCORE': 'cxd867d7843d6a489b23392719435c70ec3442161a'}, 'governance': {'zip': '../../core_contracts/governance.zip', 'SCORE': 'cx53a614e3d3ce7d8ee01920cba52d9c8f53fd0f6d'}, 'oracle': {'zip': '../../core_contracts/oracle.zip', 'SCORE': 'cxed97bdb35a7ca1b3993e400e4dba9e11610338f7'}, 'sicx': {'zip': '../../token_contracts/sicx.zip', 'SCORE': 'cx313a7b07cd2cb470f6baaf4aaccecf925456a3ca'}, 'bnUSD': {'zip': '../../token_contracts/bnUSD.zip', 'SCORE': 'cxb0e4035799bc4e61230713a6fedde99924524736'}, 'bnXLM': {'zip': '../../token_contracts/bnXLM.zip', 'SCORE': 'cx9e4e0161c712e36c86d7a85f8c39ce45a5616944'}, 'bnDOGE': {'zip': '../../token_contracts/bnDOGE.zip', 'SCORE': 'cxed2a1ca56302958c5aa084191a367d374337e32c'}, 'baln': {'zip': '../../token_contracts/baln.zip', 'SCORE': 'cx6c096b9e451d1f7e5f6e4016a992d2a892a9053a'}, 'bwt': {'zip': '../../token_contracts/bwt.zip', 'SCORE': 'cx43e5e722f6607396696f3a57e9bd98ea4f21b5ac'}}

In [120]:
# Cell 5a
# The following addresses are those deployed to the private tbears server.

contracts = {'loans': {'zip': 'core_contracts/loans.zip',
  'SCORE': 'cxa0f715fb2c4bc8f4c6399c2cc26167a27be0aa61'},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': 'cxbabed822d59b605dbeb6322735c529b292baac3b'},
 'dividends': {'zip': 'core_contracts/dividends.zip',
  'SCORE': 'cx1379084f45776301abda3849c6e374f460ee0155'},
 'reserve': {'zip': 'core_contracts/reserve.zip',
  'SCORE': 'cx71dda2221bf88faddc8f84b72ffd6db296e5609e'},
 'daofund': {'zip': 'core_contracts/daofund.zip',
  'SCORE': 'cxfd09787f23d23b945fa0c7eb55b5aa69425da1c8'},
 'rewards': {'zip': 'core_contracts/rewards.zip',
  'SCORE': 'cx27aa3bf62145822e60d85fa5d18dabdcff5b9ada'},
 'dex': {'zip': 'core_contracts/dex.zip',
  'SCORE': 'cx01eee12b6614e5328e0a84261652cb7f055e0176'},
 'governance': {'zip': 'core_contracts/governance.zip',
  'SCORE': 'cxd7b3e71dcff3d75392216e208f28ef68e8a54ec0'},
 'oracle': {'zip': 'core_contracts/oracle.zip',
  'SCORE': 'cxed97bdb35a7ca1b3993e400e4dba9e11610338f7'},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': 'cx799f724e02560a762b5f2bd3b6d2d8d59d7aecc1'},
 'bnUSD': {'zip': 'token_contracts/bnUSD.zip',
  'SCORE': 'cx266bdc0c35828c8130cdf1cbaa3ad109f7694722'},
 'bnXLM': {'zip': 'token_contracts/bnXLM.zip',
  'SCORE': 'cx266bdc0c35828c8130cdf1cbaa3ad109f7694722'},
 'bnDOGE': {'zip': 'token_contracts/bnDOGE.zip',
  'SCORE': 'cx266bdc0c35828c8130cdf1cbaa3ad109f7694722'},
 'baln': {'zip': 'token_contracts/baln.zip',
  'SCORE': 'cx4d0768508a7ff550de4405f27aebfb8831565c19'},
 'bwt': {'zip': 'token_contracts/bwt.zip',
  'SCORE': 'cx663f9d59163846d9f6c6f7b586858c59aa8878a9'}}

In [121]:
# Cell 6
# Define deploy and send_tx functions

def send_icx(to, amount, wallet):
    transaction = TransactionBuilder()\
        .from_(wallet.get_address())\
        .to(to)\
        .value(amount)\
        .step_limit(1000000) \
        .nid(NID) \
        .nonce(2) \
        .version(3) \
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    return icon_service.send_transaction(signed_transaction)


def compress():
    """
    Compress all SCORE folders in the core_contracts and token_contracts folders.
    Make sure the oracle address is correct.
    """
    deploy = list(contracts.keys())[:]
    for directory in {"../../core_contracts", "../../token_contracts"}:
        with os.scandir(directory) as it:
            for file in it:
                archive_name = directory + "/" + file.name
                if file.is_dir() and file.name in deploy:
                    make_archive(archive_name, "zip", directory, file.name)
                    contracts[file.name]['zip'] = archive_name + '.zip'
    if network == "yeouido":
        contracts['oracle']['SCORE'] = TEST_ORACLE
    elif network == "mainnet":
        contracts['oracle']['SCORE'] = MAIN_ORACLE
                    

def deploy_SCORE(contract, params, wallet, update) -> str:
    """
    contract is of form {'zip': 'core_contracts/governance.zip', 'SCORE': 'cx1d81f93b3b8d8d2a6455681c46128868782ddd09'}
    params is a dicts
    wallet is a wallet file
    update is boolian
    """
    print(f'{contract["zip"]}')
    if update:
        dest = contract['SCORE']
    else:
        dest = GOVERNANCE_ADDRESS
    zip_file = contract['zip']
    step_limit = 3000000000
    deploy_transaction = DeployTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(dest)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content(zip_file))\
        .params(params)\
        .build()

    signed_transaction = SignedTransaction(deploy_transaction, wallet, step_limit)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    print(f'Status: {res["status"]}')
    if len(res["eventLogs"]) > 0:
        for item in res["eventLogs"]:
            print(f'{item} \n')
    if res['status'] == 0:
        print(f'Failure: {res["failure"]}')
    print('')
    return res

def send_tx(dest, value, method, params, wallet, _print = True):
    """
    dest is the name of the destination contract.
    """
    if _print:
        print('------------------------------------------------------------------------------------------------------------------')
        print(f'Calling {method}, with parameters {params} on the {dest} contract.')
        print('------------------------------------------------------------------------------------------------------------------')
    transaction = CallTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(contracts[dest]['SCORE'])\
        .value(value)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(method)\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    if _print:
        print(f'************************************************** Status: {res["status"]} **************************************************')
    if len(res["eventLogs"]) > 0:
        for item in res["eventLogs"]:
            print(f'{item} \n')
    if res['status'] == 0:
        if _print:
            print(f'Failure: {res["failure"]}')
    return res

def deploy_all(wallet, staking_wallet):
    """
    Compress and Deploy all SCOREs.
    """
    compress()

    deploy = list(contracts.keys())[:]
    deploy.remove('oracle')
    deploy.remove('staking')
    deploy.remove('sicx')
    deploy.remove('governance')
    if network == "mainnet":
        if 'bnXLM' in deploy:
            deploy.remove('bnXLM')
        if 'bnDOGE' in deploy:
            deploy.remove('bnDOGE')

    results = {}
    res = deploy_SCORE(contracts['governance'], {}, wallet, 0)
    results[f'{contracts["governance"]}|deploy|{{}}'] = res
    governance = res.get('scoreAddress', '')
    contracts['governance']['SCORE'] = governance
    params = {'_governance': governance}
    for score in deploy:
        res = deploy_SCORE(contracts[score], params, wallet, 0)
        results[f'{contracts[score]}|deploy|{params}'] = res
        contracts[score]['SCORE'] = res.get('scoreAddress', '')

    res = deploy_SCORE(contracts['staking'], {}, staking_wallet, 0)
    results[f'{contracts["staking"]}|deploy|{{}}'] = res
    contracts['staking']['SCORE'] = res.get('scoreAddress', '')

    params = {'_admin': contracts['staking']['SCORE']}
    res = deploy_SCORE(contracts['sicx'], params, staking_wallet, 0)
    results[f'{contracts["sicx"]}|deploy|{params}'] = res
    contracts['sicx']['SCORE'] = res.get('scoreAddress', '')
    
    return results

    
def config_balanced(wallet, staking_wallet):
    """
    Configure all SCOREs before launch.
    """
    config = list(contracts.keys())[:]
    config.remove('governance')
    config.remove('bnDOGE')
    config.remove('bnXLM')
    addresses = {contract: contracts[contract]['SCORE'] for contract in config}
    txns = [{'contract': 'governance', 'value': 0, 'method': 'setAddresses', 'params': {'_addresses': addresses}, 'wallet': wallet},
            {'contract': 'staking', 'value': 0, 'method': 'setSicxAddress', 'params': {'_address': contracts['sicx']['SCORE']}, 'wallet': staking_wallet},
            {'contract': 'governance', 'value': 0, 'method': 'configureBalanced', 'params': {}, 'wallet': wallet}]

    results = {}
    for tx in txns:
        res = send_tx(tx["contract"], tx["value"], tx["method"], tx["params"], tx["wallet"])
        results[f'{tx["contract"]}|{tx["method"]}|{tx["params"]}'] = res

    return results

def launch_balanced(wallet, staking_wallet):
    """
    Launch Balanced, turn on staking management, and set delegation for sICX on the Loans contract.
    """
    if network == "custom":
        preps = {
            "hx9eec61296a7010c867ce24c20e69588e2832bc52",  # ICX Station
            "hx000e0415037ae871184b2c7154e5924ef2bc075e"}  # iBriz-ICONOsphere
    elif network == "yeouido":
        preps = {
            "hx23823847f593ecb65c9e1ea81a789b02766280e8",  # ICX Station
            "hxe0cde6567eb6529fe31b0dc2f2697af84847f321",  # iBriz-ICONOsphere
            "hx83c0fc2bcac7ecb3928539e0256e29fc371b5078",  # Mousebelt
            "hx48b4636e84d8c491c88c18b65dceb7598c4600cc",  # Parrot 9
            "hxb4e90a285a79687ec148c29faabe6f71afa8a066"}  # ICONDAO
    elif network == "mainnet":
        preps = {
            "hxfba37e91ccc13ec1dab115811f73e429cde44d48",  # ICX Station
            "hx231a795d1c719b9edf35c46b9daa4e0b5a1e83aa",  # iBriz-ICONOsphere
            "hxbc9c73670c79e8f6f8060551a792c2cf29a8c491",  # Mousebelt
            "hx28c08b299995a88756af64374e13db2240bc3142"}  # Parrot 9
    else:
        return

    txns = [{'contract': 'governance', 'value': 0, 'method': 'launchBalanced', 'params': {}, 'wallet': wallet},
            {'contract': 'staking', 'value': 0, 'method': 'toggleStakingOn', 'params': {}, 'wallet': staking_wallet},
            {'contract': 'governance', 'value': 0, 'method': 'delegate', 'params': {'_delegations': [{'_address': prep, '_votes_in_per': str(100 * ICX // len(preps))} for prep in preps]}, 'wallet': wallet}]

    results = {}
    for tx in txns:
        res = send_tx(tx["contract"], tx["value"], tx["method"], tx["params"], tx["wallet"])
        results[f'{tx["contract"]}|{tx["method"]}|{tx["params"]}'] = res

    return results
   

def get_scores_json(contracts):
    """
    Prints out dictionary of SCORE addresses for use in testing UI.
    """
    scores = {}
    for score in contracts:
        scores[score] = contracts[score]['SCORE']
    return json.dumps(scores)

def call_tx(dest: str, method: str, params: dict = {}, _print = True):
    """
    dest is the name of the destination contract.
    """
    if _print:
        print('------------------------------------------------------------------------------------------------------------------')
        print(f'Reading {method}, with parameters {params} on the {dest} contract.')
        print('------------------------------------------------------------------------------------------------------------------')
    call = CallBuilder()\
        .from_(wallet.get_address())\
        .to(contracts[dest]['SCORE'])\
        .method(method)\
        .params(params)\
        .build()
    result = icon_service.call(call)
    if _print:
        print(result)
    return result

In [122]:
# Cell 7
# Deploy and configure Balanced. Print results if anything goes wrong.

if network == 'custom':
    confirm = 'Yes'
else:
    confirm = input(f'Deploying Balanced to {network}. Proceed (Yes/No)? ')
if confirm == 'Yes':
    results = {}
    deploy_all(btest_wallet, staking_wallet)
    print('------------------------------------------------------------------------------------------------------------------')
    print(contracts)
    print('----------Contracts for Testing UI--------------------------------------------------------------------------------')
    print(get_scores_json(contracts))

../../core_contracts/governance.zip


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
Status: 1

../../core_contracts/loans.zip


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
Status: 1

../../core_contracts/dividends.zip


...

Retrying in 1 seconds...
Status: 1

../../token_contracts/bwt.zip


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
Status: 1

../../core_contracts/staking.zip


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
Status: 1

../../token_contracts/sicx.zip
Status: 1

------------------------------------------------------------------------------------------------------------------
{'loans': {'zip': '../../core_contracts/loans.zip', 'SCORE': 'cxac3e578fabe6e85fd16e0a56c4aba5e4ef6c5b07'}, 'staking': {'zip': '../../core_contracts/staking.zip', 'SCORE': 'cxfd6c291c153448ba0f2e240de4070ed4e66ea806'}, 'dividends': {'zip': '../../core_contracts/dividends.zip', 'SCORE': 'cx1d0ad8a419e064d9c19c852dff201aaf8e7db64e'}, 'reserve': {'zip': '../../core_contracts/reserve.zip', 'SCORE': 'cx075a56b059e3d531aac7ed1f44b6d9cc1abcc884'}, 'daofund': {'zip': '../../core_contracts/daofund.zip', 'SCORE': 'cxb8f2d7c28362d7336d5da58f3a8b6869987d3b78'}, 'rewards': {'zip': '../../core_contracts/rewards.zip', 'SCORE': 'cx70a08b6d45f0d6b14f1607203b803a0726e1ddeb'}, 'dex': {'zip': '../../core_contracts/dex.zip', 'SCORE': 'cx010e0976dd26604f4e3951a39c9401dc46347731'}, 'governance': {'zip': '../../core_con

In [123]:
# Cell 7a
# Configure Balanced

config_results = config_balanced(btest_wallet, staking_wallet)
print(config_results)

------------------------------------------------------------------------------------------------------------------
Calling setAddresses, with parameters {'_addresses': {'loans': 'cxac3e578fabe6e85fd16e0a56c4aba5e4ef6c5b07', 'staking': 'cxfd6c291c153448ba0f2e240de4070ed4e66ea806', 'dividends': 'cx1d0ad8a419e064d9c19c852dff201aaf8e7db64e', 'reserve': 'cx075a56b059e3d531aac7ed1f44b6d9cc1abcc884', 'daofund': 'cxb8f2d7c28362d7336d5da58f3a8b6869987d3b78', 'rewards': 'cx70a08b6d45f0d6b14f1607203b803a0726e1ddeb', 'dex': 'cx010e0976dd26604f4e3951a39c9401dc46347731', 'oracle': 'cxed97bdb35a7ca1b3993e400e4dba9e11610338f7', 'sicx': 'cx670fb2edb40949a44300254d9f7600392d6fd9f7', 'bnUSD': 'cx76cf21c4fb573d81801b7dccd70b550f10479982', 'baln': 'cx7d17124b21d6b06876f37428fbc6b100af4bde32', 'bwt': 'cx966f4be71c5b4253a8539b10df657c1b80f418e5'}} on the governance contract.
------------------------------------------------------------------------------------------------------------------


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
************************************************** Status: 1 **************************************************
------------------------------------------------------------------------------------------------------------------
Calling setSicxAddress, with parameters {'_address': 'cx670fb2edb40949a44300254d9f7600392d6fd9f7'} on the staking contract.
------------------------------------------------------------------------------------------------------------------
************************************************** Status: 1 **************************************************
------------------------------------------------------------------------------------------------------------------
Calling configureBalanced, with parameters {} on the governance contract.
------------------------------------------------------------------------------------------------------------------


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cxac3e578fabe6e85fd16e0a56c4aba5e4ef6c5b07', 'indexed': ['AssetAdded(Address,str,bool)', 'cx670fb2edb40949a44300254d9f7600392d6fd9f7', 'sICX', '0x1'], 'data': []} 

{'scoreAddress': 'cxac3e578fabe6e85fd16e0a56c4aba5e4ef6c5b07', 'indexed': ['AssetAdded(Address,str,bool)', 'cx76cf21c4fb573d81801b7dccd70b550f10479982', 'bnUSD', '0x0'], 'data': []} 

{'scoreAddress': 'cxac3e578fabe6e85fd16e0a56c4aba5e4ef6c5b07', 'indexed': ['AssetAdded(Address,str,bool)', 'cx7d17124b21d6b06876f37428fbc6b100af4bde32', 'BALN', '0x1'], 'data': []} 

{"governance|setAddresses|{'_addresses': {'loans': 'cxac3e578fabe6e85fd16e0a56c4aba5e4ef6c5b07', 'staking': 'cxfd6c291c153448ba0f2e240de4070ed4e66ea806', 'dividends': 'cx1d0ad8a419e064d9c19c852dff201aaf8e7db64e', 'reserve': 'cx075a56b059e3d531aac7ed1f44b6d9cc1abcc884', 'daofund': 'cxb8f2d7c28362d7336d5da58f3a8b6

In [124]:
# Cell 7b
# Launch Balanced
# We may want to make this a payable method and have the governance SCORE borrow bnUSD,
# start and name the sICXbnUSD market, and add it as a rewards DataSource.

launch_results = launch_balanced(btest_wallet, staking_wallet)
print(launch_results)

------------------------------------------------------------------------------------------------------------------
Calling launchBalanced, with parameters {} on the governance contract.
------------------------------------------------------------------------------------------------------------------


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cxac3e578fabe6e85fd16e0a56c4aba5e4ef6c5b07', 'indexed': ['ContractActive(str,str)', 'Loans'], 'data': ['Active']} 

------------------------------------------------------------------------------------------------------------------
Calling toggleStakingOn, with parameters {} on the staking contract.
------------------------------------------------------------------------------------------------------------------


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
************************************************** Status: 1 **************************************************
------------------------------------------------------------------------------------------------------------------
Calling delegate, with parameters {'_delegations': [{'_address': 'hx9eec61296a7010c867ce24c20e69588e2832bc52', '_votes_in_per': '50000000000000000000'}, {'_address': 'hx000e0415037ae871184b2c7154e5924ef2bc075e', '_votes_in_per': '50000000000000000000'}]} on the governance contract.
------------------------------------------------------------------------------------------------------------------


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
************************************************** Status: 1 **************************************************
{'governance|launchBalanced|{}': {'txHash': '0x36f050831f103ea309b51c42f694d8e178be75498f9840987f4ab1610e8f1cf1', 'blockHeight': 563975, 'blockHash': '0xe050378f4fbd2a4971211ed243527448551900b2ba207a080b9269561ab1f7a3', 'txIndex': 1, 'to': 'cx87a679f6a69db8c84552a86926055c3741e894ad', 'stepUsed': 417520, 'stepPrice': 10000000000, 'cumulativeStepUsed': 417520, 'eventLogs': [{'scoreAddress': 'cxac3e578fabe6e85fd16e0a56c4aba5e4ef6c5b07', 'indexed': ['ContractActive(str,str)', 'Loans'], 'data': ['Active']}], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 \x00\x00\x00\x10\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x80\

In [154]:
# Cell 8
# Deploy or Update a single SCORE

contract_name = 'dex'
update = 1
params = {}
if update == 0:
    params = {'_governance': contracts['governance']['SCORE']}

compress()
contract = contracts[contract_name]
confirm = input(f'{"Updating" if update else "Deploying"} {contract_name} with params: {params} to {network}. Proceed (Yes/No)? ')
if confirm == 'Yes':
    deploy_SCORE(contract, params, btest_wallet, update)

Updating dex with params: {} to custom. Proceed (Yes/No)? Yes
../../core_contracts/dex.zip


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
Status: 1



In [125]:
for dict in data:
    for items in dict.items():
        print(items[0])
        send_tx('loans', 300*ICX, 'depositAndBorrow', {'_asset': 'bnUSD', '_amount': 50* ICX}, KeyWallet.load(bytes.fromhex(items[1])))


hxc92735c9a954399c2fb722ae28bf22a10e27491e
------------------------------------------------------------------------------------------------------------------
Calling depositAndBorrow, with parameters {'_asset': 'bnUSD', '_amount': 50000000000000000000} on the loans contract.
------------------------------------------------------------------------------------------------------------------


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cxac3e578fabe6e85fd16e0a56c4aba5e4ef6c5b07', 'indexed': ['ICXTransfer(Address,Address,int)', 'cxac3e578fabe6e85fd16e0a56c4aba5e4ef6c5b07', 'cxfd6c291c153448ba0f2e240de4070ed4e66ea806', '0x1043561a8829300000'], 'data': []} 

{'scoreAddress': 'cx670fb2edb40949a44300254d9f7600392d6fd9f7', 'indexed': ['Mint(Address,int,bytes)', 'cxac3e578fabe6e85fd16e0a56c4aba5e4ef6c5b07'], 'data': ['0x1043561a8829300000', '0x4e6f6e65']} 

{'scoreAddress': 'cx670fb2edb40949a44300254d9f7600392d6fd9f7', 'indexed': ['Transfer(Address,Address,int,bytes)', 'hx0000000000000000000000000000000000000000', 'cxac3e578fabe6e85fd16e0a56c4aba5e4ef6c5b07', '0x1043561a8829300000'], 'data': ['0x4e6f6e65']} 

{'scoreAddress': 'cxfd6c291c153448ba0f2e240de4070ed4e66ea806', 'indexed': ['TokenTransfer(Address,int,str)', 'cxac3e578fabe6e85fd16e0a56c4aba5e4ef6c5b07', '0x1043561

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cxac3e578fabe6e85fd16e0a56c4aba5e4ef6c5b07', 'indexed': ['ICXTransfer(Address,Address,int)', 'cxac3e578fabe6e85fd16e0a56c4aba5e4ef6c5b07', 'cxfd6c291c153448ba0f2e240de4070ed4e66ea806', '0x1043561a8829300000'], 'data': []} 

{'scoreAddress': 'cx670fb2edb40949a44300254d9f7600392d6fd9f7', 'indexed': ['Mint(Address,int,bytes)', 'cxac3e578fabe6e85fd16e0a56c4aba5e4ef6c5b07'], 'data': ['0x1043561a8829300000', '0x4e6f6e65']} 

{'scoreAddress': 'cx670fb2edb40949a44300254d9f7600392d6fd9f7', 'indexed': ['Transfer(Address,Address,int,bytes)', 'hx0000000000000000000000000000000000000000', 'cxac3e578fabe6e85fd16e0a56c4aba5e4ef6c5b07', '0x1043561a8829300000'], 'data': ['0x4e6f6e65']} 

{'scoreAddress': 'cxfd6c291c153448ba0f2e240de4070ed4e66ea806', 'indexed': ['TokenTransfer(Address,int,str)', 'cxac3e578fabe6e85fd16e0a56c4aba5e4ef6c5b07', '0x1043561

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cxac3e578fabe6e85fd16e0a56c4aba5e4ef6c5b07', 'indexed': ['ICXTransfer(Address,Address,int)', 'cxac3e578fabe6e85fd16e0a56c4aba5e4ef6c5b07', 'cxfd6c291c153448ba0f2e240de4070ed4e66ea806', '0x1043561a8829300000'], 'data': []} 

{'scoreAddress': 'cx670fb2edb40949a44300254d9f7600392d6fd9f7', 'indexed': ['Mint(Address,int,bytes)', 'cxac3e578fabe6e85fd16e0a56c4aba5e4ef6c5b07'], 'data': ['0x1043561a8829300000', '0x4e6f6e65']} 

{'scoreAddress': 'cx670fb2edb40949a44300254d9f7600392d6fd9f7', 'indexed': ['Transfer(Address,Address,int,bytes)', 'hx0000000000000000000000000000000000000000', 'cxac3e578fabe6e85fd16e0a56c4aba5e4ef6c5b07', '0x1043561a8829300000'], 'data': ['0x4e6f6e65']} 

{'scoreAddress': 'cxfd6c291c153448ba0f2e240de4070ed4e66ea806', 'indexed': ['TokenTransfer(Address,int,str)', 'cxac3e578fabe6e85fd16e0a56c4aba5e4ef6c5b07', '0x1043561

In [103]:
send_tx('loans', 500*ICX, 'depositAndBorrow', {'_asset': 'bnUSD', '_amount': 50* ICX}, btest_wallet)


------------------------------------------------------------------------------------------------------------------
Calling depositAndBorrow, with parameters {'_asset': 'bnUSD', '_amount': 50000000000000000000} on the loans contract.
------------------------------------------------------------------------------------------------------------------


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cx21dcfb066413047fc8a05d1ed0496c8341cb9d00', 'indexed': ['ICXTransfer(Address,Address,int)', 'cx21dcfb066413047fc8a05d1ed0496c8341cb9d00', 'cx10404e91d3306c2739b05f14581e9a84f63fe08b', '0x1b1ae4d6e2ef500000'], 'data': []} 

{'scoreAddress': 'cxf4a2e3410e095de77993263166b1b0ca7c7b012b', 'indexed': ['Mint(Address,int,bytes)', 'cx21dcfb066413047fc8a05d1ed0496c8341cb9d00'], 'data': ['0x1b1ae4d6e2ef500000', '0x4e6f6e65']} 

{'scoreAddress': 'cxf4a2e3410e095de77993263166b1b0ca7c7b012b', 'indexed': ['Transfer(Address,Address,int,bytes)', 'hx0000000000000000000000000000000000000000', 'cx21dcfb066413047fc8a05d1ed0496c8341cb9d00', '0x1b1ae4d6e2ef500000'], 'data': ['0x4e6f6e65']} 

{'scoreAddress': 'cx10404e91d3306c2739b05f14581e9a84f63fe08b', 'indexed': ['TokenTransfer(Address,int,str)', 'cx21dcfb066413047fc8a05d1ed0496c8341cb9d00', '0x1b1ae4d

{'txHash': '0xb285a52bb618a0c0184be3ae6c46f2b3021d6028230064d2c044536018a9b7b1',
 'blockHeight': 561458,
 'blockHash': '0x0eb270a76beec5f45eb412d8ac98919ba27857813b90898f13f2ce2f0df28cd4',
 'txIndex': 1,
 'to': 'cx21dcfb066413047fc8a05d1ed0496c8341cb9d00',
 'stepUsed': 772400,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 772400,
 'eventLogs': [{'scoreAddress': 'cx21dcfb066413047fc8a05d1ed0496c8341cb9d00',
   'indexed': ['ICXTransfer(Address,Address,int)',
    'cx21dcfb066413047fc8a05d1ed0496c8341cb9d00',
    'cx10404e91d3306c2739b05f14581e9a84f63fe08b',
    '0x1b1ae4d6e2ef500000'],
   'data': []},
  {'scoreAddress': 'cxf4a2e3410e095de77993263166b1b0ca7c7b012b',
   'indexed': ['Mint(Address,int,bytes)',
    'cx21dcfb066413047fc8a05d1ed0496c8341cb9d00'],
   'data': ['0x1b1ae4d6e2ef500000', '0x4e6f6e65']},
  {'scoreAddress': 'cxf4a2e3410e095de77993263166b1b0ca7c7b012b',
   'indexed': ['Transfer(Address,Address,int,bytes)',
    'hx0000000000000000000000000000000000000000',
    'cx21dc

In [146]:
for dict in data:
    for items in dict.items():
        print(items[0])
        call_tx('loans', 'getAccountPositions', {'_owner': items[0]})


hxc92735c9a954399c2fb722ae28bf22a10e27491e
------------------------------------------------------------------------------------------------------------------
Reading getAccountPositions, with parameters {'_owner': 'hxc92735c9a954399c2fb722ae28bf22a10e27491e'} on the loans contract.
------------------------------------------------------------------------------------------------------------------
{'pos_id': '0x1', 'created': '0x5c12d8d0266f7', 'address': 'hxc92735c9a954399c2fb722ae28bf22a10e27491e', 'snap_id': '0xf5', 'snaps_length': '0x1', 'last_snap': '0xf5', 'first day': '0xf5', 'assets': {'sICX': '0x1043561a8829300000', 'bnUSD': '0x2bcd40a70853a0000'}, 'total_debt': '0x1a3107502244a0ee3', 'collateral': '0x1043561a8829300000', 'ratio': '0x89dfa3baa1c89284', 'standing': 'Mining'}
hxa9e67162f41c05c651d50d1197fc8596864af801
------------------------------------------------------------------------------------------------------------------
Reading getAccountPositions, with parameters {'_own

In [140]:
call_tx('loans', 'getAccountPositions', {'_owner': btest_wallet.get_address()})


------------------------------------------------------------------------------------------------------------------
Reading getAccountPositions, with parameters {'_owner': 'hx3f01840a599da07b0f620eeae7aa9c574169a4be'} on the loans contract.
------------------------------------------------------------------------------------------------------------------
{'message': 'That address has no outstanding loans or deposited collateral.'}


{'message': 'That address has no outstanding loans or deposited collateral.'}

In [151]:
wallets = []
for dict in data:
    for items in dict.items():
        wallets.append(items[0])
params = {'_holders': wallets}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['rewards']['SCORE'])\
                    .method("getBalnHoldings")\
                    .params(params)\
                    .build()
result = icon_service.call(call)
result

{'hxc92735c9a954399c2fb722ae28bf22a10e27491e': '0x0',
 'hxa9e67162f41c05c651d50d1197fc8596864af801': '0x0',
 'hx7b4dd3efcc72c5e2b210852742593178c210742c': '0x0',
 'hx22722df8f35ddbe40954cb4496619ae932a4ac3b': '0x0',
 'hxe6645482385915b2329f444f59abf49fe5024bd7': '0x0'}

In [156]:
# Check BALN that has been distributed on the rewards SCORE, and to platform contract addresses.

borrowerCount = int(call_tx('loans', 'borrowerCount', {}, False), 0)

addresses = []
for i in range(1, borrowerCount + 1):
    position = call_tx('loans', 'getPositionByIndex', {'_index': i, '_day': -1}, False)
    addresses.append(position['address'])

holders = call_tx('rewards', 'getBalnHoldings', {'_holders': addresses}, False)

total_balances = 0
baln_balances = {}
for contract in ['rewards', 'reserve', 'bwt', 'dex', 'daofund']:
    result = int(call_tx('baln', 'balanceOf', {'_owner': contracts[contract]['SCORE']}, False), 0)
    baln_balances[contract] = result / 10**18
    total_balances += result

i = 0
holdings = {i: [key, int(holders[key], 0), int(holders[key], 0) / 10**18] for i, key in enumerate(holders.keys())}
total = 0
for key in holdings:
    total += holdings[key][1]
    print(f'{holdings[key]}')

print(f'Total unclaimed: {total / 10**18}')
print(baln_balances)
print(f'Total BALN: {total_balances / 10**18}')

['hxc92735c9a954399c2fb722ae28bf22a10e27491e', 0, 0.0]
['hxa9e67162f41c05c651d50d1197fc8596864af801', 0, 0.0]
['hx7b4dd3efcc72c5e2b210852742593178c210742c', 0, 0.0]
['hx22722df8f35ddbe40954cb4496619ae932a4ac3b', 0, 0.0]
['hxe6645482385915b2329f444f59abf49fe5024bd7', 0, 0.0]
['hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db', 0, 0.0]
['hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', 0, 0.0]
Total unclaimed: 0.0
{'rewards': 700000.0, 'reserve': 100000.0, 'bwt': 400000.0, 'dex': 0.0, 'daofund': 800000.0}
Total BALN: 2000000.0


In [155]:
# Create a new pool and generate assets for it

sICX_amount = 16 * ICX
bnUSD_amount = 50 * ICX
icx_price = 2
sICX_contribute_amount = int(10 * ICX)
bnUSD_contribute_amount = int(25 * ICX)

txns = [{'contract': 'staking', 'value': sICX_amount, 'method': 'stakeICX', 'params': {}},
        {'contract': 'loans', 'value': int(icx_price * 6 * bnUSD_amount), 'method': 'depositAndBorrow', 'params': {'_asset': 'bnUSD', '_amount': bnUSD_amount}},
        {'contract': 'sicx', 'value': 0, 'method': 'transfer', 'params': {'_to': contracts['dex']['SCORE'], '_value': sICX_amount, '_data': json.dumps({"method": "_deposit"}).encode()}},
        {'contract': 'bnUSD', 'value': 0, 'method': 'transfer', 'params': {'_to': contracts['dex']['SCORE'], '_value': bnUSD_amount, '_data': json.dumps({"method": "_deposit"}).encode()}},
        {'contract': 'dex', 'value': 0, 'method': 'add', 'params': {'_baseToken': contracts['sicx']['SCORE'], '_quoteToken': contracts['bnUSD']['SCORE'], '_baseValue': sICX_contribute_amount, '_quoteValue': bnUSD_contribute_amount}}]

supply_liquidity_results = {}
for tx in txns:
    res = send_tx(tx["contract"], tx["value"], tx["method"], tx["params"], wallet)
    supply_liquidity_results[f'{tx["contract"]}|{tx["method"]}|{tx["params"]}'] = res

------------------------------------------------------------------------------------------------------------------
Calling stakeICX, with parameters {} on the staking contract.
------------------------------------------------------------------------------------------------------------------
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cx670fb2edb40949a44300254d9f7600392d6fd9f7', 'indexed': ['Mint(Address,int,bytes)', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb'], 'data': ['0xde0b6b3a76400000', '0x4e6f6e65']} 

{'scoreAddress': 'cx670fb2edb40949a44300254d9f7600392d6fd9f7', 'indexed': ['Transfer(Address,Address,int,bytes)', 'hx0000000000000000000000000000000000000000', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', '0xde0b6b3a76400000'], 'data': ['0x4e6f6e65']} 

{'scoreAddress': 'cxfd6c291c153448ba0f2e240de4070ed4e66ea806', 'indexed': ['TokenTransfer(Address,int,str)', 'hxe7af5fcfd8dfc67530a01a0e40388268

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cxac3e578fabe6e85fd16e0a56c4aba5e4ef6c5b07', 'indexed': ['ICXTransfer(Address,Address,int)', 'cxac3e578fabe6e85fd16e0a56c4aba5e4ef6c5b07', 'cxfd6c291c153448ba0f2e240de4070ed4e66ea806', '0x2086ac351052600000'], 'data': []} 

{'scoreAddress': 'cx670fb2edb40949a44300254d9f7600392d6fd9f7', 'indexed': ['Mint(Address,int,bytes)', 'cxac3e578fabe6e85fd16e0a56c4aba5e4ef6c5b07'], 'data': ['0x2086ac351052600000', '0x4e6f6e65']} 

{'scoreAddress': 'cx670fb2edb40949a44300254d9f7600392d6fd9f7', 'indexed': ['Transfer(Address,Address,int,bytes)', 'hx0000000000000000000000000000000000000000', 'cxac3e578fabe6e85fd16e0a56c4aba5e4ef6c5b07', '0x2086ac351052600000'], 'data': ['0x4e6f6e65']} 

{'scoreAddress': 'cxfd6c291c153448ba0f2e240de4070ed4e66ea806', 'indexed': ['TokenTransfer(Address,int,str)', 'cxac3e578fabe6e85fd16e0a56c4aba5e4ef6c5b07', '0x2086ac3

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cx76cf21c4fb573d81801b7dccd70b550f10479982', 'indexed': ['Transfer(Address,Address,int,bytes)', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', 'cx010e0976dd26604f4e3951a39c9401dc46347731', '0x2b5e3af16b1880000'], 'data': ['0x7b226d6574686f64223a20225f6465706f736974227d']} 

{'scoreAddress': 'cx7d17124b21d6b06876f37428fbc6b100af4bde32', 'indexed': ['Mint(Address,int,bytes)', 'cx70a08b6d45f0d6b14f1607203b803a0726e1ddeb'], 'data': ['0x152d02c7e14af6800000', '0x4e6f6e65']} 

{'scoreAddress': 'cx7d17124b21d6b06876f37428fbc6b100af4bde32', 'indexed': ['Transfer(Address,Address,int,bytes)', 'hx0000000000000000000000000000000000000000', 'cx70a08b6d45f0d6b14f1607203b803a0726e1ddeb', '0x152d02c7e14af6800000'], 'data': ['0x4e6f6e65']} 

{'scoreAddress': 'cx7d17124b21d6b06876f37428fbc6b100af4bde32', 'indexed': ['Transfer(Address,Address,int,bytes)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
************************************************** Status: 0 **************************************************
Failure: {'code': 32, 'message': 'Balanced DEX Rewards distribution in progress, please try again shortly'}
